BOW(Bag of Words)의 치명적 단점: 단어가 사용된 횟수를 기반으로 벡터를 만들기 때문에 **단어의 순서나 문맥을 활용하지 못함**

<br/>

# N-gram
### 'N개의 연속적인 단어들의 나열'을 의미, 하나의 토큰이 두 개 이상의 단어로 구성될 수 있음. 
### N이 바로 토큰을 구성하는 단어의 수를 나타냄
ex) "The future depends on what we do in the present"를 토큰화 <br/>
&nbsp; &nbsp; &nbsp; &nbsp; bi-gram(N이 2일 때) -> 'The future', 'future depends', 'depends on', 'on what', ... 'the present'


### 일반적으로 N이 커질수록 더 많은 정보를 담을 수 있음
### 그러나, BOW 방식은 기본적으로 과적합 문제가 발생하므로 N은 3까지 사용하는 것이 적당

In [1]:
# 20 뉴스그룹 데이터 가져오기
from sklearn.datasets import fetch_20newsgroups

# 무교, 종교, 그래픽, 우주 카테고리에 해당하는 포스트 가져옴
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

news_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories)
news_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=categories)

In [2]:
# train/test split
X_train = news_train.data
y_train = news_train.target

X_test = news_test.data
y_test = news_test.target

In [4]:
# 1. Unigram(N이 1개)으로 TfidfVectorizer 생성
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

cachedStopWords = stopwords.words('english')

tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}", # 3글자 이상 문자 추출
                        decode_error = 'ignore',
                        lowercase=True,
                        stop_words=cachedStopWords,
                        max_df=0.5,
                        min_df=2)  

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 11483)


In [7]:
# N-gram 사용으로 변수가 늘어나 과적합 우려가 있으므로
# 릿지 회귀 사용
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier()

ridge.fit(X_train_tfidf, y_train)

print("Train score: {:.3f}".format(ridge.score(X_train_tfidf, y_train)))
print("Test score: {:.3f}".format(ridge.score(X_test_tfidf, y_test)))

Train score: 0.976
Test score: 0.766


In [8]:
# bi-gram
# TfidfVectorizer의 경우 매개변수로 ngram_range를 사용하면 됨

tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}", # 3글자 이상 문자 추출
                        decode_error = 'ignore',
                        lowercase=True,
                        stop_words=cachedStopWords,
                        ngram_range=(1, 2),            # unigram, bi-gram 모두 사용
                        max_df=0.5,
                        min_df=2) 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)
# 특성의 수가 11483 -> 26550으로 두 배 이상 증가함

(2034, 26550)


In [13]:
# bi-gram 특성 출력하기

bigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 1]
print("bi-gram features: ", bigram_features[:50])
print()

# 릿지 회귀
ridge.fit(X_train_tfidf, y_train)

print("Train score: {:.3f}".format(ridge.score(X_train_tfidf, y_train)))
print("Test score: {:.3f}".format(ridge.score(X_test_tfidf, y_test)))

bi-gram features:  ["'cause can't", "'em better", "'expected errors'", "'karla' next", "'nodis' password", "'official doctrine", "'ok see", "'sci astro'", "'what's moonbase", 'aas american', 'ability means', 'ability pass', 'able accept', 'able afford', 'able control', 'able convince', 'able draw', 'able establish', 'able find', 'able get', 'able help', 'able import', 'able judge', 'able make', 'able read', 'able run', 'able see', 'able support', 'able tell', 'able upgrade', 'able use', 'able view', 'able work', 'abolish law', 'abortion services', 'abraham moses', 'absence belief', 'absolute moral', 'absolute morality', 'absolute objective', 'absolute sense', 'absolute truth', 'absolutely nothing', 'abstact submission', 'abstract videotape', 'abstracts authors', 'abstracts files', 'abuse mismanagement', 'acad alaska', 'academic institutions']

Train score: 0.976
Test score: 0.773


In [15]:
# tri-gram
tfidf = TfidfVectorizer(token_pattern="[a-zA-Z']{3,}", # 3글자 이상 문자 추출
                        decode_error = 'ignore',
                        lowercase=True,
                        stop_words=cachedStopWords,
                        ngram_range=(1, 3),            # unigram, bi-gram, tri-gram 모두 사용
                        max_df=0.5,
                        min_df=2) 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.shape)

(2034, 32943)


In [17]:
# tri-gram 특성 출력
trigram_features = [f for f in tfidf.get_feature_names_out() if len(f.split()) > 2]
print("tri-gram features: ", trigram_features[:50])
print()

# 릿지 회귀
ridge.fit(X_train_tfidf, y_train)

print("Train score: {:.3f}".format(ridge.score(X_train_tfidf, y_train)))
print("Test score: {:.3f}".format(ridge.score(X_test_tfidf, y_test)))

tri-gram features:  ["'em better shots", "'expected errors' basically", "'karla' next one", "'nodis' password also", "'official doctrine think", "'ok see warning", "'what's moonbase good", 'aas american astronautical', 'ability means infallible', 'able accept donations', 'able afford stuff', 'able tell liar', 'abolish law prophets', 'absence belief existence', 'absolute moral code', 'abstact submission deadline', 'abstract videotape robert', 'abstracts authors submit', 'abuse mismanagement contact', 'acad alaska edu', 'accept donations keep', 'acceptable range cmd', 'acceptable range orbiter', 'acceptable range rpm', 'acceptance sent may', 'accepted doctrine clearly', 'accepted give number', 'accepted presentations published', 'access contact nasa', 'access digex com', 'access extra ram', 'access huge astronomical', 'access roms data', 'access visa mastercard', 'accessed hours day', 'accidentally set blaze', 'accompanied offsetting reductions', 'accomplish exploration saturnian', 'acco